# Memory Context Provider

In this notebook, you'll use **Neo4j Agent Memory** as a context provider for the Microsoft Agent Framework. Unlike the knowledge graph providers in Lab 6 that retrieve from a static dataset, agent memory enables persistent conversation history, entity extraction, and preference learning.

You'll learn about the three memory types:
- **Short-term**: Recent conversation history
- **Long-term**: Entities, preferences, and facts
- **Reasoning**: Similar past task traces

***

Load the environment variables and import the required modules.

In [ ]:
import sys
sys.path.insert(0, '../shared')

import asyncio

from pydantic import SecretStr

from agent_framework.azure import AzureAIClient
from azure.identity.aio import AzureCliCredential

from neo4j_agent_memory import MemoryClient, MemorySettings
from neo4j_agent_memory.integrations.microsoft_agent import (
    Neo4jContextProvider,
    Neo4jMicrosoftMemory,
)

from config import get_agent_config, Neo4jConfig

## Configure Memory Settings

The `MemorySettings` configures the connection to Neo4j and the embedding provider. We'll use the same Neo4j instance and Azure AI endpoint from the workshop environment.

In [ ]:
# Get workshop configuration
config = get_agent_config()
neo4j_config = Neo4jConfig()

# Configure memory settings
settings = MemorySettings(
    neo4j={
        "uri": neo4j_config.uri,
        "username": neo4j_config.username,
        "password": SecretStr(neo4j_config.password),
    },
)

print(f"Neo4j URI: {neo4j_config.uri}")
print(f"Memory settings configured")

## Create the Memory Client and Unified Memory

The `Neo4jMicrosoftMemory` class provides a unified interface that combines:
- A `Neo4jContextProvider` (automatically injects context before agent runs)
- A `Neo4jChatMessageStore` (persists conversation history)
- Direct access to all three memory types

In [ ]:
# Create memory client
memory_client = MemoryClient(settings)
await memory_client.__aenter__()

# Create unified memory interface
session_id = "workshop-demo"

memory = Neo4jMicrosoftMemory.from_memory_client(
    memory_client=memory_client,
    session_id=session_id,
    include_short_term=True,
    include_long_term=True,
    include_reasoning=True,
    extract_entities=True,
)

print(f"Session: {session_id}")
print(f"Memory created with context provider and chat store")

> The `Neo4jMicrosoftMemory` creates both a context provider (`memory.context_provider`) and a chat message store (`memory.chat_store`). The context provider implements `BaseContextProvider` and runs automatically.

***

## Create and Run the Agent

Attach the memory context provider to the agent. The provider will:
1. **Before each run**: Retrieve relevant conversation history, entities, and preferences
2. **After each run**: Save messages and extract entities from the conversation

In [ ]:
async def run_conversation():
    async with AzureCliCredential() as credential:
        async with AzureAIClient(
            project_endpoint=config.project_endpoint,
            model_deployment_name=config.model_name,
            credential=credential,
        ) as client:
            agent = client.as_agent(
                name="workshop-memory-agent",
                instructions=(
                    "You are a helpful assistant with persistent memory. "
                    "You can remember previous conversations and user preferences. "
                    "When you notice the user expressing a preference, acknowledge it."
                ),
                context_providers=[memory.context_provider],
            )
            session = agent.create_session()

            # First interaction
            queries = [
                "Hi! I'm interested in learning about Apple's products.",
                "What about their risk factors? I'm particularly concerned about supply chain issues.",
                "Can you remind me what we discussed about Apple?",
            ]

            for query in queries:
                print(f"User: {query}\n")
                print("Assistant: ", end="", flush=True)

                response = await agent.run(query, session=session)
                print(response.text)
                print("\n" + "-" * 50 + "\n")

    await asyncio.sleep(0.1)

await run_conversation()

> Notice how the agent's responses become more contextually aware as the conversation progresses. The memory context provider injects relevant conversation history before each response.

***

## Explore Memory Contents

Let's look at what the memory system has stored.

In [ ]:
# Check what's in memory
results = await memory.search_memory(
    query="Apple products and risks",
    include_messages=True,
    include_entities=True,
    include_preferences=True,
    limit=5,
)

print("=== Memory Search Results ===\n")

if results.get("messages"):
    print(f"Messages found: {len(results['messages'])}")
    for msg in results["messages"][:3]:
        print(f"  [{msg['role']}] {msg['content'][:100]}...")
    print()

if results.get("entities"):
    print(f"Entities found: {len(results['entities'])}")
    for entity in results["entities"][:5]:
        print(f"  {entity['name']} ({entity['type']}): {entity.get('description', 'N/A')[:80]}")
    print()

if results.get("preferences"):
    print(f"Preferences found: {len(results['preferences'])}")
    for pref in results["preferences"][:3]:
        print(f"  [{pref['category']}] {pref['preference']}")

***

[View the complete code](../financial_data_load/solution_srcs/07_01_memory_context_provider.py)

[Move on to the Memory Tools Agent Notebook](02_memory_tools_agent.ipynb)

In [ ]:
# Cleanup
await memory_client.__aexit__(None, None, None)